# SuAVE Image to Label SVM Prediction Model

In [27]:
%%javascript
function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("survey_url='".concat(getQueryStringValue("surveyurl")).concat("'"));
IPython.notebook.kernel.execute("views='".concat(getQueryStringValue("views")).concat("'"));
IPython.notebook.kernel.execute("view='".concat(getQueryStringValue("view")).concat("'"));
IPython.notebook.kernel.execute("user='".concat(getQueryStringValue("user")).concat("'"));
IPython.notebook.kernel.execute("csv_file='".concat(getQueryStringValue("csv")).concat("'")); 
IPython.notebook.kernel.execute("dzc_file='".concat(getQueryStringValue("dzc")).concat("'")); 
IPython.notebook.kernel.execute("params='".concat(getQueryStringValue("params")).concat("'")); 
IPython.notebook.kernel.execute("active_object='".concat(getQueryStringValue("activeobject")).concat("'")); 
IPython.notebook.kernel.execute("full_notebook_url='" + window.location + "'"); 

<IPython.core.display.Javascript object>

### Import all packages

In [28]:
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')  # "error", "ignore", "always", "default", "module" or "once"

# More imports
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import csv
import pandas as pd
import re
import cv2
import os

# import the necessary packages for SVM predictor
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import imutils
import joblib

# import histogram func
from histogram import Histograms

# Import widget functionality
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

## Run and choose the column name that coresponds with the column label to be predicted

In [29]:
# use csv file to grab images/labels
csv_path = "/home/jovyan/temp_csvs/" + csv_file
df = pd.read_csv(csv_path)

localdzc = dzc_file.replace("https://maxim.ucsd.edu/dzgen/lib-staging-uploads","/lib-nfs/dzgen")
full_images_location = localdzc.replace("/content.dzc","/full_images/")

from bs4 import BeautifulSoup
from urllib.parse import urljoin

response = requests.get(full_images_location)
soup = BeautifulSoup(response.text, 'html.parser')

image_links = soup.find_all('a', href=True)

image_urls = []

def get_string_before_last_period(input_string):
    parts = input_string.rsplit('.', 1)
    if len(parts) > 1:
        return parts[0]
    else:
        return input_string

images_in_df = list(df["#img"].unique())

for link in image_links:
    image_url = link['href']
    if get_string_before_last_period(image_url) in images_in_df:
    
        if not image_url.startswith(('http:', 'https:')):
            full_image_url = urljoin(full_images_location, image_url)
        else:
            full_image_url = image_url

        image_urls.append(full_image_url)

# Choose column of label for prediction
toPredict = list(df.columns.values)

pred_menu = {}
for i in range(0, len(toPredict)):
    pred_menu[toPredict[i]] = toPredict[i]

def f(predictions_menu):
    return predictions_menu

out2 = interact(f, predictions_menu=pred_menu)

interactive(children=(Dropdown(description='predictions_menu', options={'Name': 'Name', 'OAID#link#multi': 'OA…

## Choose type of histogram to extract from images

In [30]:
hist_type = { 
    'HSV Color Histogram': 0,
    'Blue Histogram': 1,
    'Red Histogram': 2,
    'Green Histogram': 3
    }

def f(histogram_type):
    return histogram_type
typeH= interact(f, histogram_type=hist_type);

interactive(children=(Dropdown(description='histogram_type', options={'HSV Color Histogram': 0, 'Blue Histogra…

## Get histogram from each image in the dataset

In [31]:
# init the image suffix
suffix = '.jpg'
labels = []

labelHeader = out2.widget.result

# grab chosen column names
nameCol = df['#img']
predCol = df[labelHeader]

# add all fabric columns to the y set
for i in range (0,len(predCol)):
    labels.append(predCol[i])
# create new hist_list
hist_list = []
# gather images from path created from file names in csv file
for i in range(len(nameCol)):
    image_name = nameCol[i]
    url = full_images_location + image_name + ".png"
    response = requests.get(url)

    if response.status_code == 200:
        # Read the image from the response content
        image_bytes = np.frombuffer(response.content, np.uint8)
        im = cv2.imdecode(image_bytes, cv2.IMREAD_COLOR)
    else:
        print(f"Failed to fetch image from URL: {url}")
    if typeH == 0:
        hist = Histograms.extract_color_histogram(im)
    elif typeH == 1: 
        hist = Histograms.extract_blue_histogram(im)
    elif typeH == 2: 
        hist = Histograms.extract_red_histogram(im)
    elif typeH == 3: 
        hist = Histograms.extract_green_histogram(im)
    hist_list.append(hist)
    
# transform labels into numerical system
le = LabelEncoder()
labels = le.fit_transform(predCol)



# separate data into test/train sets for features/labels
(xtrain, xtest, ytrain, ytest) = train_test_split(np.array(hist_list),labels, test_size = 0.5)

# Train the linear regression classifier
model2 = LinearSVC()
model2.fit(xtrain, ytrain)

# Calculate predictions on the data set
predictions = model2.predict(np.array(hist_list))
target_names = [str(class_name) for class_name in le.classes_]

print(classification_report(labels, predictions, target_names=target_names))

                                                     precision    recall  f1-score   support

                                           A Olioso       0.00      0.00      0.00         2
                                        A. Alassane       0.00      0.00      0.00         1
                                          A. Aureli       0.00      0.00      0.00         3
                                           A. Boone       0.00      0.00      0.00         1
                                          A. Bouvet       0.00      0.00      0.00         1
                                         A. Croteau       0.00      0.00      0.00         1
                                         A. Dausman       0.00      0.00      0.00         1
                                           A. Gomez       0.00      0.00      0.00         1
                                        A. Guendouz       0.00      0.00      0.00         2
                                     A. Mahamat Ali       0.00      0

In [36]:
# save the label
filePath = "labels/"
fileName = labelHeader + '_' + csv_file[:-4] + '_svmModel' + '.txt'
file = open(os.path.join(filePath, fileName),"a+") 
file.write(str(typeH))
file.write(out2.widget.result)
file.close()

#Generate model file and save

modelPath = "models/"
modelName =  user + "_svm_" + out2.widget.result + "_" + str(typeH) + ".pkl"
joblib.dump(model2, os.path.join(modelPath, modelName)) 

['models/joeykaminsky2_svm_Name_0.pkl']

In [37]:
typeH

0

In [38]:
labelHeader

'Name'

In [39]:
labelHeader + '_' + csv_file[:-4] + 'svmModel' + '.txt'

'Name_joeykaminsky2_Tester_13svmModel.txt'